# Crawling German police press releases regarding NYE firworks incidents

## Idee
Jedes Jahr gibt es um den 31. Dezember hunderte durch Feuerwerkskörper verursachte Einsätze der Polizei und Rettungskräfte. Ich sammle alle Presseberichte, die die Polizeibehörden bei presseportal.de/blaulicht veröffentlichen, um sie dann zu aggregieren und auszuwerten.

## Technik
Zum crawlen Verwende ich Selenium und BeautifulSoup, um auch JavaScript basierte Seiten einfach zu parsen. 

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime
import platform
import time
from bs4 import BeautifulSoup

Der Crawler sucht auf der Website nach einem zu bestimmenden HTML Tag und liefert den Inhalt als String zurück. Um verschiedene Anti-Bot Maßnahmen zu umgehen, braucht man manchmal einen realistischen User-Agent und künstliche Wartezeiten.

In [2]:
def crawl(url, classname):
    path = './chromedriver' if platform.system() == "Darwin" else './chromedriver.exe'
        
    service = Service(path)
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--window-size=1920,1080')
    user_agent = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36'
    options.add_argument(f'user-agent={user_agent}')
    driver = webdriver.Chrome(service=service, options=options)
    try:
        driver.get(url)
        time.sleep(5)
        content = driver.find_element(By.CLASS_NAME, classname)
        text = content.get_attribute('innerHTML')
        driver.quit()
        return text
    except Exception as e:
        print(f"Chrome Error: {str(e)}")
        error_file = open('error.html', 'w')
        error_file.write(driver.find_element(By.TAG_NAME, 'html').get_attribute('innerHTML'))
        error_file.close()
        driver.quit()

Ein Artikel besteht aus der ID, dem Veröffentlichungsdatum, der Behörde, dem Titel und dem Inhalt. Diese werden über die einzelnen HTML Tags und Klassen extrahiert und formatiert.

Das fertige Objekt hat folgendes Format:

    {
        id: <ID>,
        title: <Titel des Artikels>,
        date: <Datum der Veröffentlichung>,
        location: <Behörde>,
        pol_id: <ID der Behörde (für spätere Zurordung)>,
        content: <Inhalt des Artikels>
    }

In [3]:
def find_articles(text):
        soup = BeautifulSoup(text, 'html.parser')
        articles = soup.find_all('article', class_="news")
        ll = []
        for article in articles:
            id = article.get('data-label')
            datestring = article.find('div',class_='date').text
            date = datetime.strptime(datestring.replace(' – ', ' '),"%d.%m.%Y %H:%M")
            pol = article.find('p',class_='customer')
            pol_id = pol.find('a').get('data-url').split('/')[-1]
            location = pol.text
            title = article.find('h3').text
            content = article.find_all('p')[-1].text
            ll.append({'id': id,'date': date,'location': location,'behoerde': pol_id, 'title': title, 'content': content})
        print(f"found {len(ll)} articles")
        return ll

Es gibt eine Übersichtsseite aller Behörden, die auf presseportal.de/blaulicht veröffentlichen. Sie sind hier nach Bundesland sortiert. Ich crawle sie alle, damit ich später die Behörden aus den Artikeln einem Bundesland zuordnen kann

In [4]:
def find_police_ids(text):
    soup = BeautifulSoup(text, 'html.parser')
    sections = soup.find_all('div',class_='row')
    results = []
    for s in sections[2:]:
        try:
            bundesland = s.find('h3',class_='dienststellen-headline').text
            elements = s.find_all('div',class_='col')
            for element in elements:
                name = element.find('a').text
                id = element.find('a').get('href').split('/')[-1]
                label = element.find('div').text
                results.append({'id': id,'bundesland': bundesland,'label': label,'name': name})
        except Exception as e :
            print(e)
            print(s)
    print(f"found {len(results)} police ids")
    return results


Da immer 30 Artikel pro Seite angezeigt werden, muss man mehrmals crawlen, um ältere Meldungen zu erhalten. Die neusten 30 Artikel findet man auf der Seite https://presseportal.de/blaulicht/0 und die danach sind auf der Seite mit dem Suffix 30, 60, 90, usw. abrufbar.

In [10]:

raw_texts = []
for n in range(10):
    raw_texts.append(crawl(f'https://www.presseportal.de/blaulicht/{n*30}', 'storyliste-wrapper'))


In [11]:
meldungen = []
for text in raw_texts:
    articles = find_articles(text)
    meldungen.extend(articles)

found 30 articles
found 30 articles
found 30 articles
found 30 articles
found 30 articles
found 30 articles
found 30 articles
found 30 articles
found 30 articles
found 30 articles


In [12]:
text_polizei = crawl('https://www.presseportal.de/blaulicht/p_dienststellen.htx', 'dienststellen-container')
police_ids = find_police_ids(text_polizei)

found 522 police ids


Die Behörden ID eines Artikels wird mit der Liste aller Behörden abgeglichen, um relevante Informationen wie das Bundesland dem Artikel anzuhängen

In [13]:
def get_police(id,police_ids):
    for police in police_ids:
        if id in police['id']:
            return police
    return None

Mittels einer Wortliste wird nach relevanten Artikeln gesucht.

In [14]:
word_list = ['feuerwerkskörper', 'knaller', 'böller', 'boeller', 'rakete', 'feuerwerk','explosion','detonation']
positive_meldungen = []

for meldung in meldungen:
    for word in word_list:
        if word in meldung['content'].lower():
            if meldung not in positive_meldungen:
                positive_meldungen.append(meldung)
                meldung['behoerde'] = get_police(meldung['behoerde'],police_ids)
                print(meldung)

{'id': '5682466', 'date': datetime.datetime(2023, 12, 31, 11, 19), 'location': 'Bundespolizeiinspektion Ebersbach', 'behoerde': {'id': '74161', 'bundesland': 'Sachsen', 'label': 'Bundespolizeiinspektion Ebersbach', 'name': 'Ebersbach '}, 'title': 'BPOLI EBB: Feuerwerk mit 420 Gramm Explosivstoff beschlagnahmt', 'content': 'Zittau (ots) - 29.12.2023 / 22:30 Uhr / Zittau Ein am 29. Dezember 2023 bei Zittau aus Polen einreisender Mann wurde durch die Bundespolizei mit einer Feuerwerksbatterie gestoppt, die es in sich hat. Der 35-jährige Vietnamese wurde um 22:30 Uhr in der Grenzkontrollstelle auf der B 178n kontrolliert. Er hatte vermutlich in der Tschechischen Republik Pyrotechnik der Kategorie F4 gekauft. Zwar handelt es sich bei diesem ...'}
{'id': '5682462', 'date': datetime.datetime(2023, 12, 31, 11, 7), 'location': 'Polizeiinspektion Osnabrück', 'behoerde': {'id': '104236', 'bundesland': 'Niedersachsen', 'label': 'Polizeiinspektion', 'name': 'Osnabrück '}, 'title': 'POL-OS: Osnabrüc